### Week 5 Day 4

AutoGen Core - Distributed

I'm only going to give a Teaser of this!!

Partly because I'm unsure how relevant it is to you. If you'd like me to add more content for this, please do let me know..

In [1]:
from dataclasses import dataclass
from autogen_core import AgentId, MessageContext, RoutedAgent, message_handler
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.messages import TextMessage
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.tools.langchain import LangChainToolAdapter
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.agents import Tool
from IPython.display import display, Markdown

from dotenv import load_dotenv

load_dotenv(override=True)

ALL_IN_ONE_WORKER = False

### Start with our Message class

In [2]:

@dataclass
class Message:
    content: str

### And now - a host for our distributed runtime

In [12]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntimeHost

host = GrpcWorkerAgentRuntimeHost(address="localhost:50051")
host.start() 

### Let's reintroduce a tool

In [13]:
serper = GoogleSerperAPIWrapper()
langchain_serper =Tool(name="internet_search", func=serper.run, description="Useful for when you need to search the internet")
autogen_serper = LangChainToolAdapter(langchain_serper)

In [14]:
instruction1 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons in favor of choosing AutoGen; the pros of AutoGen."

instruction2 = "To help with a decision on whether to use AutoGen in a new AI Agent project, \
please research and briefly respond with reasons against choosing AutoGen; the cons of Autogen."

judge = "You must make a decision on whether to use AutoGen for a project. \
Your research team has come up with the following reasons for and against. \
Based purely on the research from your team, please respond with your decision and brief rationale."

### And make some Agents

In [15]:
class Player1Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Player2Agent(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client, tools=[autogen_serper], reflect_on_tool_use=True)

    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        text_message = TextMessage(content=message.content, source="user")
        response = await self._delegate.on_messages([text_message], ctx.cancellation_token)
        return Message(content=response.chat_message.content)
    
class Judge(RoutedAgent):
    def __init__(self, name: str) -> None:
        super().__init__(name)
        model_client = OpenAIChatCompletionClient(model="gpt-4o-mini")
        self._delegate = AssistantAgent(name, model_client=model_client)
        
    @message_handler
    async def handle_my_message_type(self, message: Message, ctx: MessageContext) -> Message:
        message1 = Message(content=instruction1)
        message2 = Message(content=instruction2)
        inner_1 = AgentId("player1", "default")
        inner_2 = AgentId("player2", "default")
        response1 = await self.send_message(message1, inner_1)
        response2 = await self.send_message(message2, inner_2)
        result = f"## Pros of AutoGen:\n{response1.content}\n\n## Cons of AutoGen:\n{response2.content}\n\n"
        judgement = f"{judge}\n{result}Respond with your decision and brief explanation"
        message = TextMessage(content=judgement, source="user")
        response = await self._delegate.on_messages([message], ctx.cancellation_token)
        return Message(content=result + "\n\n## Decision:\n\n" + response.chat_message.content)


In [ ]:
from autogen_ext.runtimes.grpc import GrpcWorkerAgentRuntime

if not ALL_IN_ONE_WORKER:

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()

    await Player1Agent.register(worker, "player1", lambda: Player1Agent("player1"))
    await Player2Agent.register(worker, "player2", lambda: Player2Agent("player2"))
    await Judge.register(worker, "judge", lambda: Judge("judge"))

    agent_id = AgentId("judge", "default")

else:

    worker1 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker1.start()
    await Player1Agent.register(worker1, "player1", lambda: Player1Agent("player1"))

    worker2 = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker2.start()
    await Player2Agent.register(worker2, "player2", lambda: Player2Agent("player2"))

    worker = GrpcWorkerAgentRuntime(host_address="localhost:50051")
    await worker.start()
    await Judge.register(worker, "judge", lambda: Judge("judge"))
    agent_id = AgentId("judge", "default")




AioRpcError: <AioRpcError of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "failed to connect to all addresses; last error: UNKNOWN: ipv4:127.0.0.1:50052: Failed to connect to remote host: connect: Connection refused (61)"
	debug_error_string = "UNKNOWN:Error received from peer  {created_time:"2025-05-30T16:40:47.591912+05:30", grpc_status:14, grpc_message:"failed to connect to all addresses; last error: UNKNOWN: ipv4:127.0.0.1:50052: Failed to connect to remote host: connect: Connection refused (61)"}"
>

In [8]:
response = await worker.send_message(Message(content="Go!"), agent_id)

In [9]:
display(Markdown(response.content))

## Pros of AutoGen:
Here are some reasons in favor of using AutoGen in an AI Agent project:

1. **Improved Efficiency**: AutoGen utilizes asynchronous messaging and an event-driven architecture, allowing for faster and more efficient communication among agents. This can significantly enhance the responsiveness of the AI system.

2. **Scalability**: The modular and extensible nature of AutoGen facilitates the creation of scalable systems. This means that as your project grows, it can easily adapt to increased demand or expand its capabilities without substantial rework.

3. **Customization**: With its flexible architecture, AutoGen allows for the development of tailored solutions to meet specific project needs, providing more control over the AI agent's behavior and functionality.

4. **Ease of Integration**: AutoGen can often be integrated with various tools and platforms, making it easier to incorporate into existing workflows and systems.

5. **Community and Support**: Depending on the ecosystem, AutoGen might have an active community, providing resources, documentation, and support, which can be beneficial for troubleshooting and development.

These advantages make AutoGen a strong candidate for projects requiring robust, efficient, and scalable AI agents. 

TERMINATE

## Cons of AutoGen:
Here are some cons of using AutoGen for an AI Agent project:

1. **Dependency on Input Quality**: The quality of content generated is highly contingent on the quality of input data and parameters provided. This necessitates careful selection and validation of inputs to achieve desirable outcomes.

2. **Limited Customization**: AutoGen may have limitations in tailoring models or outputs to specific needs, which could hinder the flexibility required for specialized applications.

3. **Complexity in Management**: Implementing and managing AutoGen might require additional expertise and resources, potentially complicating project workflows.

4. **Cost Implications**: Depending on the pricing model, using AutoGen could lead to increased operational costs, especially for large-scale applications.

5. **Performance Variability**: The performance can be inconsistent, influenced by various factors such as model updates and changes in underlying algorithms.

6. **Integration Challenges**: Integrating AutoGen with existing systems and workflows might introduce challenges, requiring significant development effort.

7. **Risk of Overfitting**: There is a risk that AutoGen-generated models may overfit to specific datasets, leading to reduced generalizability in real-world scenarios.

These considerations should be weighed against any potential benefits when deciding whether to use AutoGen in your project. 

TERMINATE



## Decision:

After assessing both the pros and cons of using AutoGen for the AI Agent project, I have decided to utilize AutoGen.

**Rationale**: The advantages of improved efficiency, scalability, and customization present compelling reasons to adopt AutoGen, particularly given the project's potential growth and need for tailored solutions. The community support also enhances the likelihood of successful implementation and troubleshooting. While there are valid concerns regarding input quality, management complexity, and cost, the benefits of enhanced responsiveness and the ability to scale outweigh these drawbacks, especially in a dynamic project environment.

TERMINATE

In [10]:
await worker.stop()
if not ALL_IN_ONE_WORKER:
    await worker1.stop()
    await worker2.stop()

In [11]:
await host.stop()